In [1]:
import os
import glob
import h5py
from tqdm import tqdm

import scipy.io.wavfile as wav
from scipy.fftpack import fft
from scipy.signal.windows import hann
from sklearn.linear_model import LinearRegression

import librosa

import numpy as np

import tensorflow as tf
import tensorflow_io as tfio

import warnings

2023-01-19 12:12:27.321216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 12:12:28.506713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-19 12:12:28.507087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-19 12:12:28.507108: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

### Project parameters

In [2]:
window_len = 4096                                                           # Window length in samples
hop_perc = 39.9                                                                 # Hop length relative to window length ( chosen to obtain time_samples_num = 540)
hop_len = int((hop_perc / 100.0) * window_len)     # Hop length in STFT calculation
time_samples_num = 540                                              # Number of time samples where features are calculated
mel_bands_num = 48                                                      # Number of mel bands
signal_len = 20.0                                                               # signal length in seconds
sample_rate = 44100

LMS_adj_feat = 5                                                                          # Number of adjacent features to concatenate
LMS_stride = 2                                                                              # LMS stride
feat_num = (2 * LMS_adj_feat+1) * mel_bands_num    # Number of features
f_min = 1000
f_max = int(sample_rate / 2)
tdt = 0.75                                                                                       # Time distance threshold 

In [3]:
# def _load_signal(audio_filename):
#     sample_rate, signal = wav.read(audio_filename)
#     return signal, sample_rate

# def _mix_if_necessary(signal):
#     if len(signal.shape) == 2:
#         signal = np.mean(signal, axis=1)
#     return signal

# def _cut_if_necessary(signal):
#     if signal.size / sample_rate > signal_len:
#         signal = signal[:int(sample_rate * signal_len)]
#     return signal

# def _pad_if_necessary(signal):
#     if signal.size / sample_rate < signal_len:
#         signal = np.pad(signal, int(signal_len * sample_rate - signal.size), mode='constant')
#     return signal

# def _resample_if_necessary(signal, sample_rate):
#     if sample_rate != 44100:
#         signal = librosa.resample(signal, sample_rate, 44100)
#     return signal

# def _get_num_of_frames(signal, frame_len, hop_len):
#     return int((signal.size - frame_len) / hop_len) + 1

# def _get_mel_transform_mtx(sample_rate, n_fft, n_mels, fmin, fmax):
#     return librosa.filters.mel(sr=sample_rate, n_fft=n_fft, n_mels=n_mels, fmin=fmin, fmax=fmax).T

# def _get_log_mel_spectrogram(signal, window_len, hop_len, mel_transform_mtx):
#     signal = np.pad(signal, int(window_len / 2), mode='constant')
#     window = np.hamming(window_len)
#     nb_frames = _get_num_of_frames(signal, window_len, hop_len)

#     # Calculating log mel sepctrogram with FFT
#     fft_en = np.zeros((nb_frames, int(1 + window_len / 2)))
#     lms = np.zeros((nb_frames, mel_transform_mtx.shape[1]))

#     for i in range(nb_frames):
#         signal_window = signal[i * hop_len:i * hop_len + window_len] * window
#         fft_aux = np.abs(fft(signal_window)[:1 + int(window_len / 2)])
#         fft_aux[fft_aux == 0] = np.finfo(np.float32).eps
#         fft_en[i, :] = fft_aux
#         lms[i, :] = np.dot(fft_en[i, :]**2, mel_transform_mtx)
    
#     # Managing LMS outliers with extrapolating
#     lms[lms == 0] = np.finfo(np.float32).eps
#     lms = np.log(lms.T)

#     inter_points = 7
#     mean_diff = np.mean(np.mean(lms[:, 2:5], axis=1) - np.mean(lms[:, 0:2], axis=1))
#     if mean_diff > 3:
#         for k in range(lms.shape[0]):
#             inter_val, coef, interc = _linear_interpolation(lms[k, 2:2+inter_points], 2, direction='L')
#             lms[k, 0:2] = inter_val
#     return lms

# def _linear_interpolation(array, point_no, direction='L'):
#     assert direction == 'L' or direction == 'R', "Extrapolation direction not valid"
    
#     arr_len = np.size(array)
#     x = np.linspace(0,arr_len-1,arr_len).reshape(-1,1)
#     lin_reg = LinearRegression().fit(x, array.reshape(-1,1))
#     coef = np.float(lin_reg.coef_)
#     interc = np.float(lin_reg.intercept_)
#     if direction == 'L':
#         x_extra = np.linspace(-point_no, -1, point_no)
#     else:
#         x_extra = np.linspace(arr_len, arr_len + point_no - 1, point_no)
    
#     y_extra = coef * x_extra + interc
#     return y_extra, coef, interc

# def _moving_average_filter(x, n):
#     if n % 2 == 0:
#         raise ValueError("Filter length is not odd")

#     aver = np.convolve(x, np.ones((n,)) / n, mode='same')
#     n_half = int((n - 1) / 2)
#     corr_length = np.array(range(n_half + 1, n)) / n
#     aver[:n_half] = np.divide(aver[:n_half], corr_length)
#     aver[-n_half:] = np.divide(aver[-n_half:], corr_length[::-1])

#     return aver

# def _filter_and_interpolate_lms(lms, points, step):
#     refer_points = 10
#     (lms_d0, _) = lms.shape[0], lms.shape[1]
#     lms_copy = lms.copy()
    
#     lms_curr = None
#     for k in range(points):
#         lms_temp = lms_copy if k == 0 else lms_curr
#         lms_curr = np.zeros_like(lms_copy)
#         for j in range(lms_d0):
#             arr_ext = _moving_average_filter(lms_temp[j,:refer_points], 3)       # Apply moving average filter
#             add_left, coef, interc = _linear_interpolation(arr_ext, step, 'L')          # Use linear interpolation
#             add_left = (add_left + lms_temp[j, :step]) / 2
#             lms_curr[j, :step] = add_left
#             lms_curr[j, step:] = lms_temp[j, :-step].copy()
#             lms = np.concatenate((lms_curr, lms), axis=0)
    
#     lms_curr = None
#     for k in range(points):
#         lms_temp = lms_copy if k == 0 else lms_curr
#         lms_curr = np.zeros_like(lms_copy)
#         for j in range(lms_d0):
#             arr_ext = _moving_average_filter(lms_temp[j,-refer_points:], 3)      # Apply moving average filter
#             add_right, coef, interc = _linear_interpolation(arr_ext, step, 'R')      # User linear interpolation
#             add_right = (add_right + lms_temp[j,-step:]) / 2
#             lms_curr[j, :-step] = lms_temp[j, step:].copy()
#             lms_curr[j, -step:] = add_right
#         lms = np.concatenate((lms, lms_curr), axis=0)
     
#     lms_copy = lms_curr = lms_temp = None
#     return lms

In [4]:
# warnings.simplefilter("ignore")

# item = 0
# audio_filename = audio_files[item]

# signal, sample_rate = _load_signal(audio_filename)
# signal = _mix_if_necessary(signal)
# signal = _cut_if_necessary(signal)
# signal = _pad_if_necessary(signal)
# signal = _resample_if_necessary(signal, sample_rate)

# mel_transform_mtx = _get_mel_transform_mtx(sample_rate, window_len, mel_bands_num, f_min, f_max)
# log_mel_spectrogram = _get_log_mel_spectrogram(signal, window_len, hop_len, mel_transform_mtx)
# log_mel_spectrogram = _filter_and_interpolate_lms(log_mel_spectrogram, LMS_adj_feat, LMS_stride)
# log_mel_spectrogram

# feature = log_mel_spectrogram.T
# feature.shape

In [5]:
# def _get_time_vec():
#     signal = np.pad(signal, int(window_len / 2), mode='constant')
#     nb_frames = _get_num_of_frames(signal, window_len, hop_len)
#     time_vec = np.zeros((nb_frames,))
#     for i in range(nb_frames):
#         time_vec[i] = (i * hop_len) / sample_rate
#     return time_vec

In [6]:
# item = 0
# vc_filename = vc_files[item]

# with open(vc_filename) as f:
#     minima = f.readlines()

# minima_positions = np.array([float(x.strip()) for x in minima])

# vehicle_count = 0
# if minima_positions[0] >= 0:
#     vehicle_count = minima_positions.size
# print(vehicle_count)

# minima_no = minima_positions.size
# labels_separate = np.empty((0, time_vec.size))
# for k in range(minima_no):
#     vehicle_label = tdt * np.ones_like(time_vec)
#     if minima_positions[k] >= 0:
#         abs_fun = np.abs(time_vec - minima_positions[k])
#         vehicle_label = np.fmin(vehicle_label, abs_fun)
#     labels_separate = np.append(labels_separate, vehicle_label.reshape(1, -1), axis=0)
# labels = np.min(labels_separate, axis=0)
# print(labels.shape)

In [7]:
class CreateDataset:
    def __load_signal(self, audio_filename):
        sample_rate, signal = wav.read(audio_filename)
        if signal.dtype == np.int16:
            signal = signal / 32768.0
        return signal, sample_rate

    def __mix_if_necessary(self, signal):
        if len(signal.shape) == 2:
            signal = np.mean(signal, axis=1)
        return signal

    def __cut_if_necessary(self, signal, sample_rate, signal_len):
        if signal.size / sample_rate > signal_len:
            signal = signal[:int(sample_rate * signal_len)]
        return signal

    def __pad_if_necessary(self, signal, sample_rate, signal_len):
        if signal.size / sample_rate < signal_len:
            signal = np.pad(signal, int((signal_len * sample_rate - signal.size) / 2), mode='constant')
        return signal

    def __resample_if_necessary(self, signal, sample_rate):
        if sample_rate != 44100:
            signal = librosa.resample(signal, sample_rate, 44100)
        return signal

    def __get_num_of_frames(self, signal, frame_len, hop_len):
        return int((signal.size - frame_len) / hop_len) + 1

    def __get_mel_transform_mtx(self, sample_rate, n_fft, n_mels, fmin, fmax):
        return librosa.filters.mel(sr=sample_rate, n_fft=n_fft, n_mels=n_mels, fmin=fmin, fmax=fmax).T

    def __get_log_mel_spectrogram(self, signal, window_len, hop_len, mel_transform_mtx):
        signal = np.pad(signal, int(window_len / 2), mode='constant')
        window = np.hamming(window_len)
        nb_frames = self.__get_num_of_frames(signal, window_len, hop_len)

        # Calculating log mel sepctrogram with FFT
        fft_en = np.zeros((nb_frames, int(1 + window_len / 2)))
        lms = np.zeros((nb_frames, mel_transform_mtx.shape[1]))

        for i in range(nb_frames):
            signal_window = signal[i * hop_len:i * hop_len + window_len] * window
            fft_aux = np.abs(fft(signal_window)[:1 + int(window_len / 2)])
            fft_aux[fft_aux == 0] = np.finfo(np.float32).eps
            fft_en[i, :] = fft_aux
            lms[i, :] = np.dot(fft_en[i, :]**2, mel_transform_mtx)
        
        # Managing LMS outliers with extrapolating
        lms[lms == 0] = np.finfo(np.float32).eps
        lms = np.log(lms.T)

        inter_points = 7
        mean_diff = np.mean(np.mean(lms[:, 2:5], axis=1) - np.mean(lms[:, 0:2], axis=1))
        if mean_diff > 3:
            for k in range(lms.shape[0]):
                inter_val, coef, interc = self.__linear_interpolation(lms[k, 2:2+inter_points], 2, direction='L')
                lms[k, 0:2] = inter_val
        return lms

    def __linear_interpolation(self, array, point_no, direction='L'):
        assert direction == 'L' or direction == 'R', "Extrapolation direction not valid"
        
        arr_len = np.size(array)
        x = np.linspace(0,arr_len-1,arr_len).reshape(-1,1)
        lin_reg = LinearRegression().fit(x, array.reshape(-1,1))
        coef = np.float(lin_reg.coef_)
        interc = np.float(lin_reg.intercept_)
        if direction == 'L':
            x_extra = np.linspace(-point_no, -1, point_no)
        else:
            x_extra = np.linspace(arr_len, arr_len + point_no - 1, point_no)
        
        y_extra = coef * x_extra + interc
        return y_extra, coef, interc

    def __moving_average_filter(self, x, n):
        if n % 2 == 0:
            raise ValueError("Filter length is not odd")

        aver = np.convolve(x, np.ones((n,)) / n, mode='same')
        n_half = int((n - 1) / 2)
        corr_length = np.array(range(n_half + 1, n)) / n
        aver[:n_half] = np.divide(aver[:n_half], corr_length)
        aver[-n_half:] = np.divide(aver[-n_half:], corr_length[::-1])

        return aver

    def __filter_and_interpolate_lms(self, lms, points, step):
        refer_points = 10
        (lms_d0, _) = lms.shape[0], lms.shape[1]
        lms_copy = lms.copy()
        
        lms_curr = None
        for k in range(points):
            lms_temp = lms_copy if k == 0 else lms_curr
            lms_curr = np.zeros_like(lms_copy)
            for j in range(lms_d0):
                arr_ext = self.__moving_average_filter(lms_temp[j,:refer_points], 3)       # Apply moving average filter
                add_left, coef, interc =self.__linear_interpolation(arr_ext, step, 'L')          # Use linear interpolation
                add_left = (add_left + lms_temp[j, :step]) / 2
                lms_curr[j, :step] = add_left
                lms_curr[j, step:] = lms_temp[j, :-step].copy()
            lms = np.concatenate((lms_curr, lms), axis=0)
        
        lms_curr = None
        for k in range(points):
            lms_temp = lms_copy if k == 0 else lms_curr
            lms_curr = np.zeros_like(lms_copy)
            for j in range(lms_d0):
                arr_ext = self.__moving_average_filter(lms_temp[j,-refer_points:], 3)      # Apply moving average filter
                add_right, coef, interc = self.__linear_interpolation(arr_ext, step, 'R')      # User linear interpolation
                add_right = (add_right + lms_temp[j,-step:]) / 2
                lms_curr[j, :-step] = lms_temp[j, step:].copy()
                lms_curr[j, -step:] = add_right
            lms = np.concatenate((lms, lms_curr), axis=0)
        
        lms_copy = lms_curr = lms_temp = None
        return lms
    
    def __load_audio(self, audio_filename, signal_len):
        signal, sample_rate = self.__load_signal(audio_filename)
        signal = self.__mix_if_necessary(signal)
        signal = self.__cut_if_necessary(signal, sample_rate, signal_len)
        signal = self.__pad_if_necessary(signal, sample_rate, signal_len)
        signal = self.__resample_if_necessary(signal, sample_rate)
        return signal
    
    def __get_time_vec(self, signal, window_len, hop_len, sample_rate):
        signal = np.pad(signal, int(window_len / 2), mode='constant')
        nb_frames = self.__get_num_of_frames(signal, window_len, hop_len)
        time_vec = np.zeros((nb_frames,))
        for i in range(nb_frames):
            time_vec[i] = (i * hop_len) / sample_rate
        return time_vec

    def __get_feature(self, signal, sample_rate, window_len, hop_len, mel_bands_num, f_min, f_max, adj_feat, stride):
        mel_transform_mtx = self.__get_mel_transform_mtx(sample_rate, window_len, mel_bands_num, f_min, f_max)
        log_mel_spectrogram = self.__get_log_mel_spectrogram(signal, window_len, hop_len, mel_transform_mtx)
        log_mel_spectrogram = self.__filter_and_interpolate_lms(log_mel_spectrogram, adj_feat, stride)
        feature = log_mel_spectrogram.T
        return feature

    def __get_vehicle_count(self, vc_filename):
        with open(vc_filename) as f:
            minima = f.readlines()
        minima_positions = np.array([float(x.strip()) for x in minima])
        vehicle_count = 0
        if minima_positions[0] >= 0:
            vehicle_count = minima_positions.size
        return vehicle_count
    
    def __get_label(self, vc_filename, time_vec, tdt):
        with open(vc_filename) as f:
            minima = f.readlines()

        minima_positions = np.array([float(x.strip()) for x in minima])
        minima_no = minima_positions.size
        labels_separate = np.empty((0, time_vec.size))
        for k in range(minima_no):
            vehicle_label = tdt * np.ones_like(time_vec)
            if minima_positions[k] >= 0:
                abs_fun = np.abs(time_vec - minima_positions[k])
                vehicle_label = np.fmin(vehicle_label, abs_fun)
            labels_separate = np.append(labels_separate, vehicle_label.reshape(1, -1), axis=0)
        labels = np.min(labels_separate, axis=0)
        return labels
    
    def create(self, audio_folder, signal_len, time_samples_num, feat_num, adj_feat, stride, tdt, out_dataset_name="dataset.h5"):
        audio_files = sorted(glob.glob(audio_folder + "*.wav"))
        vc_files = sorted(glob.glob(audio_folder + "*.txt"))

        files_num = len(audio_files)
        features = np.zeros((files_num, time_samples_num, feat_num))
        labels = np.zeros((files_num, time_samples_num))
        vehicle_counts = np.zeros((files_num,))

        for file_index in tqdm(range(len(audio_files))):
            audio_filename = audio_files[file_index]

            signal = self.__load_audio(audio_filename, signal_len)
            time_vec = self.__get_time_vec(signal, window_len, hop_len, sample_rate)
            features[file_index, :, :]= self.__get_feature(signal, sample_rate, window_len, hop_len, mel_bands_num, f_min, f_max, adj_feat, stride)

            vc_filename = vc_files[file_index]
            labels[file_index, :] = self.__get_label(vc_filename, time_vec, tdt)
            vehicle_counts[file_index] = self.__get_vehicle_count(vc_filename)

        hf = h5py.File(out_dataset_name, 'w')
        hf.create_dataset('features', data=features, compression="gzip")
        hf.create_dataset('labels', data=labels, compression="gzip")
        hf.create_dataset('vehicle_counts', data=vehicle_counts, compression="gzip")
        hf.close()

In [8]:
audio_folder = "../VC-PRG-1_5/"
audio_files = sorted(glob.glob(audio_folder + "*.wav"))
vc_files = sorted(glob.glob(audio_folder + "*.txt"))

warnings.simplefilter(action='ignore')
train_ds = CreateDataset()
train_ds.create(audio_folder, signal_len, time_samples_num, feat_num, LMS_adj_feat, LMS_stride, tdt, "datasets/train_dataset.h5")

100%|██████████| 250/250 [12:07<00:00,  2.91s/it]


In [11]:
audio_folder = "../VC-PRG-6/"
audio_files = sorted(glob.glob(audio_folder + "*.wav"))
vc_files = sorted(glob.glob(audio_folder + "*.txt"))

warnings.simplefilter(action='ignore')
test_ds = CreateDataset()
test_ds.create(audio_folder, signal_len, time_samples_num, feat_num, LMS_adj_feat, LMS_stride, tdt, "datasets/test_dataset.h5")

100%|██████████| 172/172 [05:42<00:00,  1.99s/it]
